### Following idea of single feature binary decoding, should look at how many sessions have more than N blocks of a rule


In [23]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
from constants.glm_constants import *
from constants.behavioral_constants import *

import seaborn as sns
import scipy.stats


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
num_bins = 2

def get_sess_beh(row, beh_path, sub):
    session = row.session_name
    behavior_path = beh_path.format(sess_name=session)
    beh = pd.read_csv(behavior_path)

    # # filter trials 
    beh = behavioral_utils.get_valid_trials(beh)
    feature_selections = behavioral_utils.get_selection_features(beh)
    beh = pd.merge(beh, feature_selections, on="TrialNumber", how="inner")
    beh = behavioral_utils.get_beliefs_per_session(beh, session, sub)
    beh = behavioral_utils.get_belief_value_labels(beh)
    beh["session"] = session
    return beh

In [27]:
sub = "SA"

if sub == "SA":
    # Monkey S
    SESSIONS_PATH = "/data/patrick_res/sessions/SA/valid_sessions.pickle"
    sessions = pd.read_pickle(SESSIONS_PATH)
    beh_path = SESS_BEHAVIOR_PATH
    all_beh = pd.concat(sessions.apply(lambda x: get_sess_beh(x, beh_path, "SA"), axis=1).values).reset_index()
elif sub == "BL":
    # Monkey B
    # all_units = pd.read_pickle("/data/patrick_res/firing_rates/BL/all_units.pickle")
    # sessions = pd.DataFrame({"session_name": all_units.session.unique()})
    sessions = pd.read_pickle("/data/patrick_res/sessions/BL/valid_sessions.pickle")
    all_beh = pd.concat(sessions.apply(lambda x: get_sess_beh(x, BL_SESS_BEHAVIOR_PATH, "BL"), axis=1).values).reset_index()
else: 
    raise ValueError()

In [28]:
block_thresh = 3
good_sess = behavioral_utils.get_good_sessions_per_rule(all_beh, block_thresh)


In [29]:
good_sess

,feat,sessions,num_essions
0,CIRCLE,"[20180615, 20180625, 20180709, 20180802, 20180...",22
1,SQUARE,"[20180615, 20180709, 20180801, 20180802, 20180...",22
2,STAR,"[20180709, 20180803, 20180817, 20180821, 20180...",15
3,TRIANGLE,"[20180615, 20180705, 20180801, 20180802, 20180...",11
4,CYAN,"[20180705, 20180801, 20180802, 20180803, 20180...",19
5,GREEN,"[20180802, 20180806, 20180808, 20180813, 20180...",19
6,MAGENTA,"[20180622, 20180705, 20180801, 20180802, 20180...",21
7,YELLOW,"[20180705, 201807250001, 20180801, 20180802, 2...",25
8,ESCHER,"[20180803, 20180808, 20180810, 20180813, 20180...",17
9,POLKADOT,"[20180705, 20180709, 20180802, 20180803, 20180...",19


In [30]:
good_sess.to_pickle(f"/data/patrick_res/sessions/{sub}/feats_at_least_{block_thresh}blocks.pickle")

### For each session, for each rule, what are the min number of trials that match: 
- high preferred features
- high conf trials where features are chosen but not preferred. 

In [8]:
def min_trials_per_session(row):
    res = []
    for sess in row.sessions:
        beh = all_beh[all_beh.session == sess]

        pref = behavioral_utils.get_chosen_preferred_single(row.feat, beh, high_val_only=False)
        min_pref = np.min(pref.groupby("Choice").count().TrialNumber)

        not_pref = behavioral_utils.get_chosen_not_preferred_single(row.feat, beh, high_val_only=False)
        min_not_pref = np.min(not_pref.groupby("Choice").count().TrialNumber)

        min_all = np.min((min_pref, min_not_pref))

        res.append({
            "feat": row.feat, 
            "session": sess, 
            "min_pref": min_pref, 
            "min_not_pref": min_not_pref,
            "min_all": min_all
        })
    return pd.DataFrame(res)
        
min_trials = pd.concat(good_sess.apply(min_trials_per_session, axis=1).values)



In [15]:
min_trials.sort_values(by="min_all")

,feat,session,min_pref,min_not_pref,min_all
4,MAGENTA,20180803,33,275,33
16,MAGENTA,20180928,34,159,34
18,GREEN,20181009,41,190,41
21,YELLOW,20181002,42,242,42
4,CIRCLE,20180803,43,270,43
...,...,...,...,...,...
7,YELLOW,20180821,152,241,152
4,CYAN,20180820,162,192,162
15,YELLOW,20180912,165,228,165
20,MAGENTA,20181005,165,250,165


,BlockNumber,PreferredBelief
39448,11,STAR
39449,11,STAR
39450,11,STAR
39451,11,RIPPLE
39452,11,RIPPLE
39453,11,MAGENTA
39454,11,MAGENTA
39455,11,MAGENTA
39456,11,MAGENTA
39457,11,MAGENTA


### Save files